In [1]:
import cv2 
import matplotlib.pyplot as plt
import os
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors
import numpy as np

In [26]:
dataset_path = "../data/fashion_small_100/"

### Feature Extraction
Extraer los features de las imagenes de la carpeta utilizando opencv con SIFT.

```keypoints, descriptors = sift.detectAndCompute(gray, None)```
- keypoints almacena las coordenadas (x,y) de cada keypoint de la imagen (esquinas, manchas, cambios bruscos de intensidad, etc).
- descriptors almacena los descriptores para cada keypoint de esa imagen (128 descriptores).

In [30]:
# Extracción de descriptores
image_descriptors = [] 
sift = cv2.SIFT_create()

# Iterar sobre cada imagen dentro de la carpeta
for filename in os.listdir(dataset_path): 
    file_path = os.path.join(dataset_path, filename)
    if os.path.isfile(file_path):
        img = cv2.imread(file_path)
        if img is None:
            print(f"No se pudo leer la imagen: {file_path}")
            continue

        # Convertir a grayscale y extraer descriptores
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        keypoints, descriptors = sift.detectAndCompute(gray, None)
        
        # TODO: Probar reducir la dimensionalidad de 128 a algo menor para más eficiencia al momento de entrenar todo el dataset

        if descriptors is not None:
            image_descriptors.append(descriptors)
        else:
            print(f"No se encontraron descriptores en: {filename}")

# K-Means para construir el diccionario visual
all_descriptors = np.vstack(image_descriptors) # Para convertir la matriz image_descriptors en un array 2D para el K-Means

# TODO: Para el dataset completo cambiar n_clusters, ver cuales valores producen los mejores resultados
#       quizas guardar objeto kmeans en un vector si tarda mucho en generar
kmeans = KMeans(n_clusters=10, random_state=0, n_init="auto").fit(all_descriptors)

# Construcción de histogramas por imagen
all_histograms = []
for descriptors in image_descriptors:
    histogram = np.zeros(kmeans.n_clusters, dtype=int)

    if descriptors is not None:
        cluster_assignments = kmeans.predict(descriptors)
        for idx in cluster_assignments:
            histogram[idx] += 1

    # Normalización
    histogram = histogram.astype(float)
    histogram /= np.sum(histogram) if np.sum(histogram) > 0 else 1
    all_histograms.append(histogram)

### Buscar imagenes más similares

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances

def create_query_histogram(image_path, kmeans): 
    sift = cv2.SIFT_create()
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    histogram = np.zeros(kmeans.n_clusters, dtype=int)

    if descriptors is not None:
        cluster_assignments = kmeans.predict(descriptors)
        for idx in cluster_assignments:
            histogram[idx] += 1

    # Normalización
    histogram = histogram.astype(float)
    histogram /= np.sum(histogram) if np.sum(histogram) > 0 else 1
    return histogram


def knn(query_histogram, database_histograms, k=5): # Calcular distancias euclidianas entre query y base de datos
    dists = euclidean_distances([query_histogram], database_histograms)[0]  
    """ 
    argsort() devuelve los índices que pondrías en orden para ordenar el array de distancias.
    Como el arreglo de distancias está en el mismo orden que las imagenes de la carpeta del dataset usamos los índices
    para acceder cada imagen directamente, sin tener que iterar linealmente sobre todos los nombres de los archivos.
    """
    knn_indices = np.argsort(dists)[:k]
    knn_distances = dists[knn_indices]

    return knn_distances, knn_indices

k = 10
image_path = "C:/Users/davie/Downloads/fashion_small/images/58282.jpg" # Path query
query_histogram = create_query_histogram(image_path, kmeans) # Procesar query
dists, indices = knn(query_histogram, all_histograms, k) # Predecir imagen más parecida del dataset
print(dists)

# Mostrar imagenes más parecidas

[0.17775723 0.19692358 0.1982169  0.21757132 0.22715112 0.23375876
 0.23507324 0.23849202 0.24375838 0.24378543]


Aplicar ponderación TF-IDF